<a href="https://colab.research.google.com/github/ravi-prakash1907/Data-Structures-and-Algo/blob/main/Submissions/Group%20Projects/Project3/English_Sentence_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 3**

## English Sentence Generator

<!--
_**([Final Report Here!!](#))**_
-->


### _Libraries_

In [17]:
from IPython.display import clear_output
import re

## Processing Steps

### Processing Level 1

1. Taking Input  
2. Converting to Lower Case  
3. Resolving for Abbriviation  
4. Removing Negation  

_(Step 1, 2, 3 and 4)_

### Processing Level 2

1. Resolving for Interrogatives  
2. Remove Stopping/Ending Symbol (. ! ? : ; etc..)

_(Step 5, 6)_

### Processing Level 3

_Split mutiple sentances on Conjunctions:_  
  - That  
  - But  
  - While  
  - Meanwhile  
  - Still
  - Before  

_Remove Interjections:_  
  - oh!  
  - yess!!  
  etc..  
  
  _(Step 7)_

In [65]:
## to remove interjections
def removeInterjections(sentance):
  wordList = sentance.split(" ")
  foundInterjections = []
  for word in wordList:
    if word[-2:] == "!!" or word[-1] == "!":
      foundInterjections.append(word)
  
  # removing found interjections
  for interjection in foundInterjections:
    wordList.remove(interjection)
  
  newSentence = listToSentance(wordList)
  return newSentence

## to extract only one sentance from grouped form
def multipleSentances(sentance):
  wordList = sentance.split(" ")
  foundConjunctions = hasWordsFrom(conjunctions, wordList)

  if foundConjunctions:
    twoSubSentences = sentance.split(foundConjunctions[0])
    twoSubSentences = removeBlanks(twoSubSentences)
    commonSub = []
    commonSub.extend(articles)
    commonSub.extend(pluralPronouns)
    commonSub.extend(singlelPronouns)
    
    hasDualSentance = False
    for thisSubSentance in twoSubSentences:
      if hasWordsFrom(allHelpingVerbs, thisSubSentance) or hasWordsFrom(commonSub, thisSubSentance):
        hasDualSentance = True
      else:
        hasDualSentance = False
      
    if hasDualSentance:
      return twoSubSentences[0]
    
  return sentance


### Processing Level 4

**Initial Splitting** _(if consists Helping Verb)_  

* ```['sub', 'HV', 'verb', 'obj']```
* ```['sub', 'HV', 'obj']```


Based on the type of sentence :  
_(except present/past simple)_

### Processing Level 5

**Initial Splitting** _(without Helping Verb)_  

* ```['sub', 'verb', 'obj']```
* ```['sub', 'obj']```

_(Step 8.2)_

### Processing Level 6

_Splitting based on the type of sentence_  

**Strusture**  

* ```['Subject', 'Helping Verb', 'Verb', 'Object']```

_(in some cases, few of the parts can be blank)_  

_(Step 8)_  

## Sentance to list creater:

```['Sun'  +  'H.V.'  +  'Verb'  +  'Obj']```

##  Processing Done in following manner

**Steps:**  
1. Get Sentance (Input)  
2. Convert to Lower/Upper Case  
3. Resolve for Abbriviation  
4. Remove Negation  
6. Remove Stopping Symbol (. ! ? : ; etc..)  
7. Split into multiple sentances on Conjunctions
8. Split the Sentance and Store in Dictionary  
  - _Parts of Sentance:_  
    a) Subject  
    b) Helping Verb  
    c) Verb  
    d) Object  



In [139]:
class sentenceSplitter:
  def __init__(self, limit = 50):
    ## global variables for class
    self.__articles = ['a', 'an', 'the']
    self.__pluralPronouns = ['some', 'any', 'these', 'their', 'our', 'those', 'them', 'they', 'you', 'i', 'we']
    self.__singlelPronouns = ['he', 'she','his', 'her', 'my', 'this', 'that', 'it']
    self.__conjunctions = ['that', 'but', 'while', 'meanwhile', 'still', 'before']
    self.__allHelpingVerbs = ['is', 'am', 'are', 'was', 'were', 'shall', 'will', 'should', 'would', 'might', 'could', 'have', 'has', 'had', 'can', 'may' 'must', 'did' 'do', 'does']


    self.__specialHV = self.__allHelpingVerbs[:7]    
    self.__firstPriority = self.__allHelpingVerbs[:5]
    self.__secondPriority = self.__allHelpingVerbs[5:11]
    self.__thirdPriority = self.__allHelpingVerbs[11:14]
    self.__forthPriority = self.__allHelpingVerbs[14:]

    ## list of lists (splitted sentances)
    self.__corpus = []
    self.__maxSize = limit   ## maximum sentences
    self.currentSize = 0


  ###########################
  ###   Private Methods   ###
  ###########################
  def __updateSize(self):
    self.currentSize = len(self.__corpus)
    return True
  
  def __isInsertionPossible(self):
    if self.__maxSize > self.currentSize:
      return True
    else:
      return False
  
  def __addInCorpus(self, formattedSentence):
    self.__corpus.append(formattedSentence)
    self.__updateSize()
    return True

  ###########################

  ## returns elements of 'allWords' that are in 'findIn'
  def __hasWordsFrom(self, allWords, findIn):
    found = []
    for searched in allWords:
      if searched in findIn:
        found.append(searched)
    if len(found) > 0:
      return found
    return False

  ## remove unnecessary space in beg or end of string and emptyStrings too
  def __removeBlanks(self, wordGroup):
    for wordIndex in range(len(wordGroup)):
      if wordGroup[wordIndex] == '':
        continue
      while wordGroup[wordIndex][0] == " ":
        wordGroup[wordIndex] = wordGroup[wordIndex][1:]
      while wordGroup[wordIndex][-1] == " ":
        wordGroup[wordIndex] = wordGroup[wordIndex][:-1]
    
    return wordGroup

  ## List of words to sentance
  def __listToSentance(self, buildingBlocks):
    buildingBlocks = self.__removeBlanks(buildingBlocks)
    newSentance = ""
    for word in buildingBlocks:
      newSentance += word + " "
    return newSentance[:-1]   # last letter is space (from for-loop)  
  
  ###########################

  ### Processing Level 4 ###
  """
  present/past continuous tense and statement
  """  
  def __commonPresentPast(self, sentance, hVerb): # 'he is boy' + pContinuous
    Subject = Object = Verb = ''
    wordWise = sentance.split(" ")
    
    indexOfHVNext = wordWise.index(hVerb)+1
    Subject = self.__listToSentance(wordWise[:indexOfHVNext-1])
    if wordWise[indexOfHVNext][-3:] == 'ing':
      Verb = wordWise[indexOfHVNext]
      if indexOfHVNext < len(wordWise):
        Object = self.__listToSentance(wordWise[indexOfHVNext+1:])
    else:
      if indexOfHVNext < len(wordWise):
        Object = self.__listToSentance(wordWise[indexOfHVNext:])
      
    return [Subject, hVerb, Verb, Object]

  """
  present/past perfect/perfect-continuous tense
  """  
  def __perfectPresentPast(self, sentance, hVerb):
    Subject = Object = Verb = ''
    wordWise = sentance.split(" ")

    gotHV = self.__hasWordsFrom(['been'], wordWise)
    if gotHV:
      indexOfHVNext = wordWise.index('been')+1
      indexOfSubNext = wordWise.index(hVerb)
      hVerb += " been"

      if wordWise[indexOfHVNext][-3:] == 'ing':
        Subject = self.__listToSentance(wordWise[:indexOfSubNext])
        Verb = wordWise[indexOfHVNext]
        if indexOfHVNext < len(wordWise):
          Object = self.__listToSentance(wordWise[indexOfHVNext+1:])
    else:
      indexOfHVNext = wordWise.index(hVerb)+1
      Subject = self.__listToSentance(wordWise[:indexOfHVNext-1])
      if indexOfHVNext <= len(wordWise):
        Verb = wordWise[indexOfHVNext]
        if indexOfHVNext < len(wordWise):
          Object = self.__listToSentance(wordWise[indexOfHVNext+1:])

    return [Subject, hVerb, Verb, Object]      

  """
  future tense and statement
  """  
  def __futureTense(self, sentance, hVerb):
    Subject = Object = Verb = ''
    wordWise = sentance.split(" ")

    if self.__hasWordsFrom(['have'], wordWise):
      gotHV = self.__hasWordsFrom(['been'], wordWise)
      if gotHV:
        indexOfHVNext = wordWise.index('been')+1
        indexOfSubNext = wordWise.index(hVerb)
        hVerb += " have been"

        if wordWise[indexOfHVNext][-3:] == 'ing':
          Subject = self.__listToSentance(wordWise[:indexOfSubNext])
          Verb = wordWise[indexOfHVNext]
          if indexOfHVNext < len(wordWise):
            Object = self.__listToSentance(wordWise[indexOfHVNext+1:]) 
      else:
        indexOfHVNext = wordWise.index('have')+1
        indexOfSubNext = wordWise.index(hVerb)
        hVerb += " have"
        
        Subject = self.__listToSentance(wordWise[:indexOfSubNext])
        Verb = wordWise[indexOfHVNext]
        if indexOfHVNext < len(wordWise):
          Object = self.__listToSentance(wordWise[indexOfHVNext+1:])  
    else:
      gotHV = self.__hasWordsFrom(['be'], wordWise)
      if gotHV:
        indexOfHVNext = wordWise.index('be')+1
        indexOfSubNext = wordWise.index(hVerb)
        hVerb += " be"
        indexOfHVNext = wordWise.index('be')+1
        if wordWise[indexOfHVNext][-3:] == 'ing':
          Subject = self.__listToSentance(wordWise[:indexOfSubNext])
          Verb = wordWise[indexOfHVNext]
          if indexOfHVNext < len(wordWise):
            Object = self.__listToSentance(wordWise[indexOfHVNext+1:])
      else:
        indexOfHVNext = wordWise.index(hVerb)+1
        Subject = self.__listToSentance(wordWise[:wordWise.index(hVerb)])
        Verb = wordWise[indexOfHVNext]
        if indexOfHVNext < len(wordWise):
          Object = self.__listToSentance(wordWise[indexOfHVNext+1:])
    
    return [Subject, hVerb, Verb, Object]

  """
  some other continuous tense and guessing statement
  """  
  def __someContinuousPlus(self, sentance, hVerb):
    Subject = Object = Verb = ''
    wordWise = sentance.split(" ")

    gotHV = self.__hasWordsFrom(['be'], wordWise)
    if gotHV:
      indexOfHVNext = wordWise.index('be')+1
      indexOfSubNext = wordWise.index(hVerb)
      hVerb += " be"

      indexOfHVNext = wordWise.index('be')+1
      if wordWise[indexOfHVNext][-3:] == 'ing':
        Subject = self.__listToSentance(wordWise[:indexOfSubNext])
        Verb = wordWise[indexOfHVNext]
        if indexOfHVNext < len(wordWise):
          Object = self.__listToSentance(wordWise[indexOfHVNext+1:]) 
    else:
      indexOfHVNext = wordWise.index(hVerb)+1
      Subject = self.__listToSentance(wordWise[:wordWise.index(hVerb)])
      Verb = wordWise[indexOfHVNext]
      if indexOfHVNext < len(wordWise):
        Object = self.__listToSentance(wordWise[indexOfHVNext+1:])
    
    return [Subject, hVerb, Verb, Object]


  ### Processing Level 5 ###
  """
  **Initial Splitting** (without Helping Verb)

  * ```['sub', 'verb', 'obj']```
  * ```['sub', 'obj']````
  """
  ###   Finishing Level 5
  def __splitSimple(self, sentance = 'the cow grazes the grass'):
    wordList = sentance.split(" ")
    totalWords = len(wordList)
    Subject = Verb = Object = ''

    if totalWords <= 2:
      Subject = wordList[0]
      if (Subject in self.__singlelPronouns) and (wordList[1][-1] == 's' and wordList[1][-2:] not in ['us', 'os', 'ss', 'is', 'as']):
        Verb = wordList[1]
      else:
        Object = wordList[1]
    else:
      # checking for Simple Past
      if sentance[-2:] == 'ed' or 'ed ' in sentance:
        for word in wordList:
          if 'ed' in word or 'ed ' in word:
            Subject = self.__listToSentance(wordList[:wordList.index(word)])
            Verb = word
            Object = self.__listToSentance(wordList[wordList.index(word)+1:])
            break
        
      # checking for Simple Present (mainly)
      else:
        sEsEndingWords = []
        for word in wordList:
          if (word[-2:] == 'es') or (word[-1] == 's' and not (word[-2:] == 'us' or word[-2:] == 'ss')):
            sEsEndingWords.append(word)

        usedArticles = self.__hasWordsFrom(self.__articles, wordList)

        # if consisits articles
        if usedArticles:
          if wordList[0] in self.__articles and len(sEsEndingWords) == 1:
            #wordList.remove(wordList[0])
            wordList[0] = self.__listToSentance(wordList[:2])
            wordList.remove(wordList[1])
            totalWords -= 1
            subEndIndex = wordList.index(sEsEndingWords[0])
            if sEsEndingWords[0] == wordList[0][len(sEsEndingWords[0]):]:
              Subject = self.__listToSentance(wordList[:subEndIndex+1])
              Verb = self.__listToSentance(wordList[subEndIndex+1:])    
              Object = self.__listToSentance(wordList[wordList.index(Verb)+1:]) 
            else:
              Subject = self.__listToSentance(wordList[:subEndIndex])
              Verb = sEsEndingWords[0]
              Object = self.__listToSentance(wordList[subEndIndex+1:])
          else:
            objBegIndex = wordList.index(usedArticles[0])
            singleSub = self.__hasWordsFrom(self.__singlelPronouns, wordList)
            if singleSub:
              subEndIndex = wordList.index(singleSub[0]) + 1
              Subject = self.__listToSentance(wordList[:subEndIndex])
              Verb = wordList[subEndIndex]
              Object = self.__listToSentance(wordList[objBegIndex:])
            else:
              Subject = self.__listToSentance(wordList[:objBegIndex-1])
              Verb = wordList[objBegIndex-1]
              Object = self.__listToSentance(wordList[objBegIndex:])
        
        else:
          if len(sEsEndingWords) == 0:
            pluralSub = self.__hasWordsFrom(self.__pluralPronouns, wordList)
            if pluralSub:
              subEndIndex = wordList.index(pluralSub[0]) + 1
            elif ' and ' in sentance:
              subEndIndex = wordList.index('and') + 1
            else:
              subEndIndex = 1
            Subject = self.__listToSentance(wordList[:subEndIndex])
            Verb = self.__listToSentance(wordList[subEndIndex:subEndIndex+1]) 
            Object = self.__listToSentance(wordList[subEndIndex+1:]) 
          else:
            if wordList.index(sEsEndingWords[0]) == 0:
              Subject = self.__listToSentance(wordList[:1])
              Verb = self.__listToSentance(wordList[1:2]) 
              Object = self.__listToSentance(wordList[2:]) 
            else:
              singleSub = self.__hasWordsFrom(self.__singlelPronouns, wordList)
              if singleSub:
                subEndIndex = wordList.index(singleSub[0]) + 1
              else:
                subEndIndex = 1
              Subject = self.__listToSentance(wordList[:subEndIndex])
              Verb = self.__listToSentance(wordList[subEndIndex:subEndIndex+1])
              
              if len(wordList) > subEndIndex+1:
                Object = self.__listToSentance(wordList[subEndIndex+1:])

    return [Subject, Verb, Object]

  

  #################################################################
  #################################################################


  ###########################
  ###   Public Methods   ###
  ###########################

  ###########################
  ### Processing Level 1 ###
  """
  1. Resolving Whitespaces and Converting to Lower Case  
  2. Resolving for Abbriviation  
  3. Removing Negation  
  """
  ###########################

  def initCleaning(self, sentance):
    sentance = re.sub(' +', ' ', sentance)
    sentance = sentance.lower()
    return sentance

  
  # resolving for abbreviations
  def resolveAbbr(self, sentance):
    if '\'' in sentance:
      splittedSentance = sentance.split(" ")

      for index in range(0, len(splittedSentance)):
        word = splittedSentance[index]

        # abbriviated 'not'
        if word[-3:] == 'n\'t':
          if word == 'can\'t':
            word = 'can not'
          elif word == 'ain\'t':
            word = 'are not'  
          elif word == 'won\'t':
            word = 'will not'
          elif word == 'shan\'t':
            word = 'shall not'
          else:
            word = word[:-3] + " not"
        
        # abbriviated 'ing'
        if word[-3:] == 'in\'':
          word = word[:-1] + 'g'
        
        # abbriviated helping verbs
        abbrHVerbs = {'m':'am', 's':'is', 're':'are', 'll':'will', 've':'have', 'd':'had'}
        if "'" in word:
          flag = False
          brokenWord = word.split('\'')
          for abbr in list(abbrHVerbs.keys()):
            if  abbr == brokenWord[1]:
              brokenWord[1] = abbrHVerbs[abbr]
              flag = True

          if flag:
            word = self.__listToSentance(brokenWord)
          else:
            word = brokenWord[0] + '\'' + brokenWord[1]
        
        splittedSentance[index] = word

      resolvedTense = self.__listToSentance(splittedSentance)
      return resolvedTense

    return sentance


  ## Removing Negation
  def negationRemover(self, sentance):
    if 'not' in sentance:
      begIndex = sentance.find("not")
      sentance = sentance[:begIndex-1] + sentance[begIndex+3:]
      sentance = self.negationRemover(sentance)
    return sentance


  ###########################
  ### Processing Level 2 ###
  """
  1. Resolving for Interrogatives  
  2. Remove Stopping/Ending Symbol (. ! ? : ; etc..)
  """
  ###########################
  
  ## Yet to be implemented
  def removeQuestion(self, sentance):
    #print("Interrogative to normal sentance converter is yet to be impleented!\n")
    return sentance

  def removeStops(self, sentance):
    if not sentance[-1].isalpha() and not sentance[-1].isdigit():
      sentance = self.removeStops(sentance[:-1])
    return sentance


  ###########################
  ### Processing Level 3 ###
  """
  1. Split mutiple sentances on Conjunctions:
    - That, But, While, Meanwhile, Still, Before etc.. 

  2. Remove Interjections:
    - oh!, yess!! etc.. 
  """
  ###########################
    
  ## to remove interjections
  def removeInterjections(self, sentance):
    wordList = sentance.split(" ")
    foundInterjections = []
    for word in wordList:
      if word[-2:] == "!!" or word[-1] == "!":
        foundInterjections.append(word)
    
    # removing found interjections
    for interjection in foundInterjections:
      wordList.remove(interjection)
    
    newSentence = self.__listToSentance(wordList)
    return newSentence

  ## to extract only one sentance from grouped form
  def multipleSentances(self, sentance):
    wordList = sentance.split(" ")
    foundConjunctions = self.__hasWordsFrom(conjunctions, wordList)

    if foundConjunctions:
      twoSubSentences = sentance.split(foundConjunctions[0])
      twoSubSentences = self.__removeBlanks(twoSubSentences)
      commonSub = []
      commonSub.extend(articles)
      commonSub.extend(pluralPronouns)
      commonSub.extend(singlelPronouns)
      
      hasDualSentance = False
      for thisSubSentance in twoSubSentences:
        if self.__hasWordsFrom(self.__allHelpingVerbs, thisSubSentance) or self.__hasWordsFrom(commonSub, thisSubSentance):
          hasDualSentance = True
        else:
          hasDualSentance = False
        
      if hasDualSentance:
        return twoSubSentences[0]
      
    return sentance


  ###########################
  ### Processing Level 6 ###
  """
  Splitting based on the type of sentence  
  ```['Subject', 'Helping Verb', 'Verb', 'Object']```

  (in some cases, few of the parts can be blank)
  """
  ###########################
  def consistsHelpingVerb(self, sentance = "he goes"):
    splittedForm = ['', '', '', '']
    wordWise = sentance.split(" ")
    gotHV = self.__hasWordsFrom(self.__firstPriority, wordWise)
    if gotHV:
      splittedForm = self.__commonPresentPast(sentance, gotHV[0])              # 'he is boy' or he is running'.
    
    elif self.__hasWordsFrom(self.__secondPriority, wordWise):
      gotHV = self.__hasWordsFrom(self.__secondPriority, wordWise)
      splittedForm = self.__futureTense(sentance, gotHV[0]) 
    
    elif self.__hasWordsFrom(self.__thirdPriority, wordWise):
      gotHV = self.__hasWordsFrom(self.__thirdPriority, wordWise)
      splittedForm = self.__perfectPresentPast(sentance, gotHV[0])             # perfect    has 
    
    elif self.__hasWordsFrom(self.__forthPriority, wordWise):
      gotHV = self.__hasWordsFrom(self.__forthPriority, wordWise)
      splittedForm = self.__someContinuousPlus(sentance, gotHV[0])       ## may be +ing
    
    else:
      HV = ''
      splittedForm = self.__splitSimple(sentance)
      splittedForm.insert(1,HV)
    
    return splittedForm



  ###########################
  ### Driver Function ###
  """
  Splitting based on the type of sentence  
  ```['Subject', 'Helping Verb', 'Verb', 'Object']```
  """
  ###########################
  def convertToList(self, textData = "It\'s a sample."):
    aSentance = self.initCleaning(textData)    
    processedTense = self.resolveAbbr(aSentance)
    processedTense = self.negationRemover(processedTense)
    cleanSentance = self.removeStops(processedTense)

    interjectionRemoval = self.removeInterjections(cleanSentance)
    extractedSentence = self.multipleSentances(interjectionRemoval)

    fullSentance = self.consistsHelpingVerb(extractedSentence) 

    self.__addInCorpus(fullSentance)
    return True



  #################################################################
  #################################################################


  
  ## Reading a sentence
  def loadSentances(self, fileName = None):
    if fileName is None:
      fileName = input("Enter the path to the text (.txt) file: ")
      if fileName[-4:] != ".txt":
        print("Invalid file path! Try next time!")
    else:
      opened = open(fileName, 'r')
      textData = opened.readlines()
      opened.close()

      if len(textData) > 0:
        for thisSentence in textData:
          if self.__isInsertionPossible():
            self.convertToList(thisSentence)
          else:
            print("50 sentences loaded!\n")
            return True
        print(len(textData),"sentences loaded!\n")
        return True
      else:
        print("Error in file! No sentence Found!!\n")
        return False


  ## Printing the available corpus
  def printSentenceList(self):
    if self.currentSize > 0:
      print("\n--------------------------\nListing the",self.currentSize,"sentances:\n--------------------------\n")
      counter = 1
      for grouppedSentence in self.__corpus:
        if counter < 10:
          print("0"+str(counter),")",end = " ")
        else:
          print(counter,")",end = " ")
        print(grouppedSentence)
        counter += 1
    else:
      print("Empty Corpus!!")


  ## get returns number of loaded available sentances



In [140]:
thisSplitter = sentenceSplitter()
thisSplitter.loadSentances("basics.txt")
thisSplitter.printSentenceList()

12 sentences loaded!


--------------------------
Listing the 12 sentances:
--------------------------

01 ) ['he', '', 'runs', 'fast']
02 ) ['he', 'is', 'running', 'fast']
03 ) ['he', 'has', 'run', 'fast']
04 ) ['he', 'has been', 'running', 'fast, since 3pm']
05 ) ['he', '', 'ran', 'fast']
06 ) ['he', 'was', 'running', 'fast']
07 ) ['he', 'had', 'run', 'fast']
08 ) ['he', 'had been', 'running', 'fast, since 3pm']
09 ) ['he', 'will', 'run', 'fast']
10 ) ['he', 'will be', 'running', 'fast']
11 ) ['he', 'will have', 'run', 'fast']
12 ) ['he', 'will have been', 'running', 'fast, since 3pm']


## Storing in Tree

**Steps:**  

# **Users' Zone**

## Main-Menu

In [144]:
def menu():
  # Printing Menu
  print("\nMain Menu:")
  print("""----------
        \n1. Load Sentences  \n2. Print List of Processed Sentences  \n3. Generate Sentences \n
        \n** Any other key to exit!!\n
        """)
  return input("Enter your choice: ")

## Main-Function


In [145]:
### To be Implemented

if __name__ == "__main__":
  
  ## Showing Menu

  while (True): 
    clear_output()
    choice = menu()
    
    print("Yet to be implemented!\nMeet Soon!!!")
    break


Main Menu:
----------
        
1. Load Sentences  
2. Print List of Processed Sentences  
3. Generate Sentences 

        
** Any other key to exit!!

        
Enter your choice: 
Yet to be implemented!
Meet Soon!!!


---

# **Experiment**

In [ ]:
3
